<a href="https://colab.research.google.com/github/NewTAs/GoogLeNet/blob/main/GoogLeNet_Step8_DriveVer_Epoch100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Step1 Code
import os
import re
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

from google.colab import drive
drive.mount('googlenet')

# New Cord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

#np.random.seed(4) # 랜덤시드 고정시키기

Drive already mounted at googlenet; to attempt to forcibly remount, call drive.mount("googlenet", force_remount=True).


In [20]:
numberOfClass = 30
pictureWeightHeightSize = 227

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TrainData_Step7/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),
        batch_size=numberOfClass,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/googlenet/MyDrive/GoogLeNet_Data/TestData_Step7/',
        target_size=(pictureWeightHeightSize, pictureWeightHeightSize),    
        batch_size=numberOfClass,
        class_mode='categorical')

Found 300 images belonging to 30 classes.
Found 60 images belonging to 30 classes.


In [21]:
input_layer = tf.keras.layers.Input(shape=(227, 227, 3))

In [22]:
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(stem_layer)
stem_layer = tf.keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(max_pool)
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(stem_layer)

In [23]:
inception_block3a_1x1 = tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(max_pool)

inception_block3a_3x3 = tf.keras.layers.Conv2D(96, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block3a_3x3 = tf.keras.layers.Conv2D(128, (3, 3), padding='SAME', activation='relu')(inception_block3a_3x3)

inception_block3a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block3a_5x5 = tf.keras.layers.Conv2D(32, (5, 5), padding='SAME', activation='relu')(inception_block3a_5x5)

inception_block3a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block3a_pool = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block3a_pool)

inception_block3a = tf.keras.layers.Concatenate()([inception_block3a_1x1, inception_block3a_3x3,
                                                   inception_block3a_5x5, inception_block3a_pool])

In [24]:
inception_block3b_1x1 = tf.keras.layers.Conv2D(32, (1, 1), strides=(1, 1), padding='SAME', activation='relu')(inception_block3a)

inception_block3b_3x3 = tf.keras.layers.Conv2D(48, (1, 1), padding='SAME', activation='relu')(inception_block3a)
inception_block3b_3x3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME', activation='relu')(inception_block3b_3x3)

inception_block3b_5x5 = tf.keras.layers.Conv2D(8, (1, 1), padding='SAME', activation='relu')(inception_block3a)
inception_block3b_5x5 = tf.keras.layers.Conv2D(16, (5, 5), padding='SAME', activation='relu')(inception_block3b_5x5)

inception_block3b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block3a)
inception_block3b_pool = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block3b_pool)

inception_block3b = tf.keras.layers.Concatenate()([inception_block3b_1x1, inception_block3b_3x3,
                                                   inception_block3b_5x5, inception_block3b_pool])

In [25]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block3b)

In [26]:
inception_block4a_1x1 = tf.keras.layers.Conv2D(192, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block4a_3x3 = tf.keras.layers.Conv2D(96, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_3x3 = tf.keras.layers.Conv2D(208, (3, 3), padding='SAME', activation='relu')(inception_block4a_3x3)

inception_block4a_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block4a_5x5 = tf.keras.layers.Conv2D(48, (5, 5), padding='SAME', activation='relu')(inception_block4a_5x5)

inception_block4a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block4a_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4a_pool)

inception_block4a = tf.keras.layers.Concatenate()([inception_block4a_1x1, inception_block4a_3x3,
                                                   inception_block4a_5x5, inception_block4a_pool])

In [27]:
inception_block4b_1x1 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_3x3 = tf.keras.layers.Conv2D(112, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_3x3 = tf.keras.layers.Conv2D(224, (3, 3), padding='SAME', activation='relu')(inception_block4b_3x3)

inception_block4b_5x5 = tf.keras.layers.Conv2D(24, (1, 1), padding='SAME', activation='relu')(inception_block4a)
inception_block4b_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4b_5x5)

inception_block4b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4a)
inception_block4b_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4b_pool)

inception_block4b = tf.keras.layers.Concatenate()([inception_block4b_1x1, inception_block4b_3x3,
                                                   inception_block4b_5x5, inception_block4b_pool])

In [28]:
inception_block4c_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4b)

inception_block4c_3x3 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4b)
inception_block4c_3x3 = tf.keras.layers.Conv2D(256, (3, 3), padding='SAME', activation='relu')(inception_block4c_3x3)

inception_block4c_5x5 = tf.keras.layers.Conv2D(24, (1, 1), padding='SAME', activation='relu')(inception_block4b)
inception_block4c_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4c_5x5)

inception_block4c_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4b)
inception_block4c_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4c_pool)

inception_block4c = tf.keras.layers.Concatenate()([inception_block4c_1x1, inception_block4c_3x3,
                                                   inception_block4c_5x5, inception_block4c_pool])

In [29]:
inception_block4d_1x1 = tf.keras.layers.Conv2D(112, (1, 1), padding='SAME', activation='relu')(inception_block4c)

inception_block4d_3x3 = tf.keras.layers.Conv2D(144, (1, 1), padding='SAME', activation='relu')(inception_block4c)
inception_block4d_3x3 = tf.keras.layers.Conv2D(288, (3, 3), padding='SAME', activation='relu')(inception_block4d_3x3)

inception_block4d_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4c)
inception_block4d_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block4d_5x5)

inception_block4d_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4c)
inception_block4d_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block4d_pool)

inception_block4d = tf.keras.layers.Concatenate()([inception_block4d_1x1, inception_block4d_3x3,
                                                   inception_block4d_5x5, inception_block4d_pool])

In [30]:
inception_block4e_1x1 = tf.keras.layers.Conv2D(256, (1, 1), padding='SAME', activation='relu')(inception_block4d)

inception_block4e_3x3 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(inception_block4d)
inception_block4e_3x3 = tf.keras.layers.Conv2D(320, (3, 3), padding='SAME', activation='relu')(inception_block4e_3x3)

inception_block4e_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(inception_block4d)
inception_block4e_5x5 = tf.keras.layers.Conv2D(128, (5, 5), padding='SAME', activation='relu')(inception_block4e_5x5)

inception_block4e_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block4d)
inception_block4e_pool = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block4e_pool)

inception_block4e = tf.keras.layers.Concatenate()([inception_block4e_1x1, inception_block4e_3x3,
                                                   inception_block4e_5x5, inception_block4e_pool])

In [31]:
max_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(inception_block4e)

In [32]:
inception_block5a_1x1 = tf.keras.layers.Conv2D(256, (1, 1), padding='SAME', activation='relu')(max_pool)

inception_block5a_3x3 = tf.keras.layers.Conv2D(160, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_3x3 = tf.keras.layers.Conv2D(320, (3, 3), padding='SAME', activation='relu')(inception_block5a_3x3)

inception_block5a_5x5 = tf.keras.layers.Conv2D(32, (1, 1), padding='SAME', activation='relu')(max_pool)
inception_block5a_5x5 = tf.keras.layers.Conv2D(128, (5, 5), padding='SAME', activation='relu')(inception_block5a_5x5)

inception_block5a_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(max_pool)
inception_block5a_pool = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block5a_pool)

inception_block5a = tf.keras.layers.Concatenate()([inception_block5a_1x1, inception_block5a_3x3,
                                                   inception_block5a_5x5, inception_block5a_pool])

In [33]:
inception_block5b_1x1 = tf.keras.layers.Conv2D(128, (1, 1), padding='SAME', activation='relu')(inception_block5a)

inception_block5b_3x3 = tf.keras.layers.Conv2D(80, (1, 1), padding='SAME', activation='relu')(inception_block5a)
inception_block5b_3x3 = tf.keras.layers.Conv2D(120, (3, 3), padding='SAME', activation='relu')(inception_block5b_3x3)

inception_block5b_5x5 = tf.keras.layers.Conv2D(16, (1, 1), padding='SAME', activation='relu')(inception_block5a)
inception_block5b_5x5 = tf.keras.layers.Conv2D(64, (5, 5), padding='SAME', activation='relu')(inception_block5b_5x5)

inception_block5b_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(inception_block5a)
inception_block5b_pool = tf.keras.layers.Conv2D(64, (1, 1), padding='SAME', activation='relu')(inception_block5b_pool)

inception_block5b = tf.keras.layers.Concatenate()([inception_block5b_1x1, inception_block5b_3x3,
                                                   inception_block5b_5x5, inception_block5b_pool])

In [34]:
avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5b) # 416
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(numberOfClass, activation='softmax')(dropout)

In [35]:
model = tf.keras.models.Model(input_layer, out_layer)

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 227, 227, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 114, 114, 64) 0           conv2d_56[0][0]                  
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 114, 114, 192 110784      max_pooling2d_13[0][0]           
____________________________________________________________________________________________

In [36]:
model.fit(train_generator, epochs=100, validation_data=test_generator)

Epoch 1/100
10/10 [==============================] - 204s 21s/step - loss: 3.4201 - accuracy: 0.0300 - val_loss: 3.4013 - val_accuracy: 0.0333
Epoch 2/100
10/10 [==============================] - 172s 17s/step - loss: 3.4015 - accuracy: 0.0200 - val_loss: 3.4013 - val_accuracy: 0.0333
Epoch 3/100
10/10 [==============================] - 170s 17s/step - loss: 3.4029 - accuracy: 0.0333 - val_loss: 3.4013 - val_accuracy: 0.0333
Epoch 4/100
10/10 [==============================] - 170s 17s/step - loss: 3.4020 - accuracy: 0.0367 - val_loss: 3.4013 - val_accuracy: 0.0333
Epoch 5/100
10/10 [==============================] - 171s 17s/step - loss: 3.4023 - accuracy: 0.0233 - val_loss: 3.4013 - val_accuracy: 0.0333
Epoch 6/100
10/10 [==============================] - 170s 17s/step - loss: 3.4018 - accuracy: 0.0367 - val_loss: 3.4015 - val_accuracy: 0.0000e+00
Epoch 7/100
10/10 [==============================] - 171s 17s/step - loss: 3.3913 - accuracy: 0.0333 - val_loss: 3.4040 - val_accuracy: 0.

In [37]:
print("-- Evaluate --")
scores = model.evaluate(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
2/2 [==============================] - 8s 4s/step - loss: 31.6470 - accuracy: 0.0000e+00
accuracy: 0.00%


In [38]:
print("-- Predict --")
output = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '4': 24, '5': 25, '6': 26, '7': 27, '8': 28, '9': 29}
[[0.000 0.000 0.000 ... 0.000 0.536 0.053]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 ...
 [0.000 0.000 0.000 ... 0.000 0.000 0.019]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.034]]


In [39]:
for i in range(0, numberOfClass):
  print("Result:", i+1)
  print(output[i])
  print("\n")

Result: 1
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.002 0.000 0.000 0.035 0.000 0.000 0.000 0.000 0.017 0.000
 0.000 0.000 0.356 0.000 0.536 0.053]


Result: 2
[0.000 0.000 0.000 0.000 0.000 0.000 0.002 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.681 0.000 0.000 0.000 0.316 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000]


Result: 3
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.955 0.000 0.000 0.045
 0.000 0.000 0.000 0.000 0.000 0.000]


Result: 4
[0.000 0.038 0.081 0.000 0.000 0.000 0.000 0.000 0.000 0.001 0.000 0.172
 0.000 0.000 0.647 0.002 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.059 0.000 0.000 0.000 0.000 0.000]


Result: 5
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.001 0.000 0.000 0.003 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.478 0.000 0.512 0.005]


Result: 6
[0.000 0.0